In [1]:
from scipy.stats import pearsonr, spearmanr
import pandas as pd

In [12]:
models = ['bloom-1b7'] # 'bloom-560m', 'bloom-1b1', 'bloom-1b7'
layers = ['inter-layer-13' , 'inter-layer-17']

# variable_list = ['ovlp-rate', 'f1-score', 'cos-similarity-nl', 'cos-similarity-code', 'xnli_acc']
langs_cos = ['ar', 'es', 'eu', 'fr', 'hi', 'pt', 'ta', 'ur', 'vi']
langs_f1 = ['ar', 'en', 'es', 'eu', 'fr', 'hi', 'mr', 'pt', 'ta', 'ur', 'vi', 'zh']
langs_ovlp = ['ar', 'eu', 'ca', 'zh', 'en', 'fr', 'hi', 'mr', 'pt', 'es', 'ta', 'ur', 'vi']
langs_acc = ['ar', 'bg', 'de', 'el', 'en', 'es', 'fr', 'hi', 'ru', 'sw', 'th', 'tr', 'ur', 'vi', 'zh']

In [3]:
def read_data_by_idx(variable, model, layer, avg=False, remove_best=False):

    if variable == 'ovlp-rate':

        if avg:  # a dict will be returned: {ckpt: avg_ovlp_rate}
            dict = {}
            with open(f'csv_files/{model}_{layer}_avg-ovlp-rate-by-ckpt.txt', 'r') as f:
                lines = f.readlines()
                for line in lines:
                    list = line.split(',')
                    dict[list[0]] = float(list[1])
                if remove_best:
                    del dict['best']
            return dict
        else:
            # in pairwise setting we cannot use multiple categories because there is a mismatch of category number between languages
            # so we use ovlp-rate for POS only
            df = pd.read_csv(f'csv_files/{model}_{layer}_ovlp-rate_POS.csv', index_col=0)
            if remove_best:
                df = df.drop(columns=['best'])
            return df 
        
    elif variable == 'f1-score':
        df = pd.read_csv(f'csv_files/{model}_f1-score.csv', index_col=0)
        if remove_best:
            df = df.drop(columns=['best'])
        return df
    
    elif variable == 'cos-similarity-nl':
        df = pd.read_csv(f'csv_files/{model}_{layer}_cos-similarity.csv', index_col=0)
        df = df.drop(index=('code'))
        if remove_best:
            df = df.drop(columns=['best'])
        return df
    
    elif variable == 'cos-similarity-code':
        df = pd.read_csv(f'csv_files/{model}_{layer}_cos-similarity.csv', index_col=0)
        dict = df.loc['code'].to_dict()
        if remove_best:
            del dict['best']
        return dict
    elif variable == 'xnli_acc':
        df = pd.read_csv(f'csv_files/{model}_xnli_acc.csv', index_col=0)
        return df

def intersection(lang_list_1, lang_list_2):
    return list(set(lang_list_1) & set(lang_list_2))

def print_pearson(variable_list_1, variable_list_2):
    pearson_corr, pearson_p_value = pearsonr(variable_list_1, variable_list_2)
    print('Pearsons correlation: %.3f' % pearson_corr)
    print('Pearsons P value: %.3f' % pearson_p_value)
    print('Pearsons P value: %.3e' % pearson_p_value)
    return pearson_corr, pearson_p_value

def print_spearman(variable_list_1, variable_list_2):
    spearman_corr, spearman_p_value = spearmanr(variable_list_1, variable_list_2)
    print('Spearmans correlation: %.3f' % spearman_corr)
    print('Spearmans P value: %.3f' % spearman_p_value)
    print('Spearmans P value: %.3e' % spearman_p_value)
    return spearman_corr, spearman_p_value


def compute_df_avg(df, lang_list):
    '''
    this function will return two lists of data extracted from df.

    returns:
        element list: each element in df, appended by columns
        avg_list: the avg element value of df through columns
    '''
    element_list = []
    avg_list = []

    ckpts = df.columns
    langs = lang_list

    for ckpt in ckpts:
        temp_list = []
        for lang in langs:
            temp_list.append(float(df[ckpt][lang]))
        
        element_list += temp_list
        avg_list.append(sum(temp_list)/len(temp_list))
    
    return element_list, avg_list


In [13]:
all_avg_1 = []
all_pairwise_1 = []
all_avg_2 = []
all_pairwise_2 = []
# variable_list = ['ovlp-rate', 'f1-score', 'cos-similarity-nl', 'cos-similarity-code']
# 'last-layer'

for model in models:
    for layer in layers:
        print("find the intersection of two language list: ")
        langs_list = intersection(langs_ovlp, langs_acc)
        print(langs_list)

        df_1 = read_data_by_idx('ovlp-rate', model, layer)
        print(df_1)
        all_avg_1 += compute_df_avg(df_1, langs_cos)[1]
        all_pairwise_1 += compute_df_avg(df_1, langs_cos)[0]
        print(all_avg_1)
        print(all_pairwise_1)

        # df_2 = read_data_by_idx('xnli_acc', model, layer)
        # print(df_2)
        # all_avg_2 += compute_df_avg(df_2, langs_list)[1]
        # all_pairwise_2 += compute_df_avg(df_2, langs_list)[0]
        # print(all_avg_2)
        # print(all_pairwise_2)
        
        # df_1 = read_data_by_idx('cos-similarity-nl', model, layer)
        # print(df_1)
        # all_avg_1 += compute_df_avg(df_1, langs_cos)[1]
        # all_pairwise_1 += compute_df_avg(df_1, langs_cos)[0]

        df_2 = read_data_by_idx('cos-similarity-nl', model, layer)
        print(df_2)
        all_avg_2 += compute_df_avg(df_2, langs_cos)[1]
        all_pairwise_2 += compute_df_avg(df_2, langs_cos)[0]
        print(all_avg_2)
        print(all_pairwise_2)

        # list_2 = read_data_by_idx('cos-similarity-code', model, 'last-layer').values()
        # print(list_2)
        # all_avg_2 += list_2
        # # no pairwise for code   

        # df_2 = read_data_by_idx('f1-score', model, layer)
        # print(df_2)
        # all_avg_2 += compute_df_avg(df_2, langs_f1)[1]
        # all_pairwise_2 += compute_df_avg(df_2, langs_f1)[0]

print("Average:")
print_pearson(all_avg_1, all_avg_2)
print_spearman(all_avg_1, all_avg_2)
print("Pairwise:")
print_pearson(all_pairwise_1, all_pairwise_2)
print_spearman(all_pairwise_1, all_pairwise_2)

find the intersection of two language list: 
['en', 'vi', 'hi', 'es', 'zh', 'ur', 'fr', 'ar']
    best  50000  100000  150000  200000
ar  0.14   0.00    0.04    0.04    0.06
ca  0.18   0.04    0.02    0.08    0.24
en  1.00   1.00    1.00    1.00    1.00
eu  0.02   0.02    0.02    0.00    0.04
fr  0.18   0.06    0.04    0.12    0.16
hi  0.10   0.04    0.06    0.08    0.08
mr  0.06   0.02    0.00    0.04    0.08
pt  0.08   0.08    0.12    0.08    0.14
es  0.14   0.06    0.08    0.14    0.18
ta  0.06   0.04    0.06    0.04    0.12
ur  0.20   0.08    0.04    0.06    0.08
vi  0.04   0.06    0.10    0.04    0.06
zh  0.10   0.06    0.08    0.06    0.08
[0.10666666666666666, 0.04888888888888889, 0.062222222222222213, 0.06666666666666668, 0.10222222222222221]
[0.14, 0.14, 0.02, 0.18, 0.1, 0.08, 0.06, 0.2, 0.04, 0.0, 0.06, 0.02, 0.06, 0.04, 0.08, 0.04, 0.08, 0.06, 0.04, 0.08, 0.02, 0.04, 0.06, 0.12, 0.06, 0.04, 0.1, 0.04, 0.14, 0.0, 0.12, 0.08, 0.08, 0.04, 0.06, 0.04, 0.06, 0.18, 0.04, 0.16, 0.0

(0.010467247396939204, 0.9219992984458723)